In [54]:
url = 'https://seller-analytics-api.wildberries.ru/api/v1/warehouse_remains?groupByBrand=True,groupBySubject=True,groupBySa=True,groupByNm=True,groupByBarcode=True,groupBySize=True'

In [55]:
import requests

In [56]:
api_key = ''

In [57]:
     headers = {
         "Authorization": api_key
     }

In [58]:
response = requests.get(url, headers=headers)

In [59]:
response

<Response [400]>

In [38]:
import json

data = json.loads(response.content)

In [71]:
task_id = data['data']['taskId']

In [72]:
data_url = f'https://seller-analytics-api.wildberries.ru/api/v1/warehouse_remains/tasks/{task_id}/download'

In [ ]:
response = requests.get(data_url, headers=headers)

In [76]:
response

<Response [200]>

In [77]:
data = json.loads(response.content)

In [ ]:
data

In [79]:
import pandas as pd

In [80]:
df = pd.DataFrame(data)

In [ ]:
df

In [84]:
df["warehouses"] = df["warehouses"].astype(str)

In [2]:
marketplace_keys = {
# TBD
 }

In [5]:
 import requests
 import pandas as pd
 from datetime import datetime


 def fetch_stocks(date_from: str, seller_legal: str, spreadsheet_id: str):
     """
     Получает данные об остатках через API Wildberries и добавляет их в Google Sheets.

     :param date_from: Дата начала выборки в формате DD.MM.YYYY.
     :param seller_legal: Юридическое лицо ("inter" или "ut").
     :param spreadsheet_id: Идентификатор Google Sheets.
     """
     # Преобразуем дату в формат YYYY-MM-DD
     try:
         formatted_date_from = datetime.strptime(date_from, "%d.%m.%Y").strftime("%Y-%m-%d")
     except ValueError as e:
         raise Exception(f"Неверный формат даты. Ожидается DD.MM.YYYY. {str(e)}")

     # Настройка URL и параметров API
     base_url = "https://statistics-api.wildberries.ru/api/v1/supplier/stocks"
     params = {
         "dateFrom": formatted_date_from
     }

     # Получаем API-ключ для текущего юридического лица
     api_key = marketplace_keys.get(seller_legal)
     if not api_key:
         raise Exception(f"API ключ для юридического лица '{seller_legal}' не найден в конфигурации.")

     headers = {
         "Authorization": api_key
     }

     # Выполняем запрос
     response = requests.get(base_url, params=params, headers=headers)

     # Проверяем статус ответа
     if response.status_code != 200:
         raise Exception(f"Ошибка API Wildberries: {response.status_code}, {response.text}")

     # Преобразуем ответ в DataFrame
     data = response.json()
     df = pd.DataFrame(data)

     # Проверяем, есть ли данные
     if df.empty:
         print("Нет данных для выбранного периода.")
         return

     # Обработка данных: добавляем юрлицо и текущую дату
     if seller_legal == "inter":
         df["legal_entity"] = "ИНТЕР"
     elif seller_legal == "ut":
         df["legal_entity"] = "АТ"
     elif seller_legal == "kravchik":
         df["legal_entity"] = "КРАВЧИК"
     else:
         df["legal_entity"] = seller_legal

     df["Дата Добавления"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

     df["period_id"] = datetime.strptime(date_from, "%d.%m.%Y").strftime("%d.%m.%Y")

     # Удаляем дубликаты по ключевым столбцам
     key_columns = ["Название склада", "Артикул продавца", "Артикул WB", "Баркод"]
     if all(col in df.columns for col in key_columns):
         df = df.drop_duplicates(subset=key_columns)

     return df


In [8]:
 try:
     df = fetch_stocks("29.11.2024", "kravchik", "1Gjx1UFXW0nZPUiDIrzNedBiJad1B__YQmTjH0uTSnws")
     print("Тест успешно завершен.")
 except Exception as e:
     print(f"Произошла ошибка: {e}")

Тест успешно завершен.


In [9]:
df

,lastChangeDate,warehouseName,supplierArticle,nmId,barcode,quantity,inWayToClient,inWayFromClient,quantityFull,category,...,brand,techSize,Price,Discount,isSupply,isRealization,SCCode,legal_entity,Дата Добавления,period_id
0,2024-11-30T10:51:05,Пушкино,AT-002G,134129266,4673737630022,0,0,1,1,Хранение вещей,...,AQRA,0,7000,90,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
1,2024-12-02T07:47:16,Коледино,SHC1-1BK,206794537,4673737631500,0,1,0,1,Мебель корпусная и мебель для хранения,...,AQRA,0,3000,66,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
2,2024-12-02T11:27:29,Коледино,HNG1-1W,157900838,4673737630152,0,2,0,2,Мебель корпусная и мебель для хранения,...,AQRA,0,5000,74,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
3,2024-12-04T12:53:35,Коледино,SHC 1-1GR,153329576,4673737630183,0,1,0,1,Мебель корпусная и мебель для хранения,...,AQRA,0,5000,81,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
4,2024-12-07T12:37:57,Электросталь,AT-002G,134129266,4673737630022,0,0,1,1,Хранение вещей,...,AQRA,0,2000,50,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
5,2024-12-08T10:59:26,Коледино,CPT 1-1BE,166735025,4673737630220,0,0,1,1,Текстиль для дома,...,AQRA,0,8650,85,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
6,2024-12-08T14:04:38,Невинномысск,HNG1-1W,157900838,4673737630152,144,0,0,144,Мебель корпусная и мебель для хранения,...,AQRA,0,5000,65,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
7,2024-12-12T10:23:46,Невинномысск,HNG1-1BK,157900837,4673737630169,142,0,0,142,Мебель корпусная и мебель для хранения,...,AQRA,0,5000,65,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
8,2024-12-12T13:21:50,Коледино,AT-001W_134125211,134125211,4673737630015,0,3,0,3,Хранение вещей,...,AQRA,0,3580,80,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
9,2024-12-12T20:41:08,Электросталь,AT-002BK,134129264,4673737630046,0,1,0,1,Хранение вещей,...,AQRA,0,2000,65,True,False,Tech,КРАВЧИК,2024-12-19 23:41:27,29.11.2024
